<a href="https://colab.research.google.com/github/melrahmtz/purple-box/blob/main/hands-on-practice/pdfparsing-docling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docling

# Parsing

In [2]:
from docling.document_converter import DocumentConverter  # For PDF content extraction

file_path = "Manuale ROTOMARR.pdf"

converter = DocumentConverter()
result = converter.convert(file_path)

structured_content = result.document.export_to_markdown()
print(structured_content)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


<!-- image -->

Revisione 00.00 - Febbraio 2015

GIRACASTAGNE AUTOMATICO ROTOMARR

<!-- image -->

## 1. INFORMAZIONI SULLA SICUREZZA

Il  presente  manuale  contiene  indicazioni  ed  informazioni fondamentali per il corretto utilizzo del GIRACASTAGNE AUTOMATICO (CUOCI CALDARROSTE) ROTOMARR .

- -Leggere il manuale  nella sua completezza per comprendere l'utilizzo della macchina;
- -Tenere questo manuale per future consultazioni in un luogo sicuro;
- -Osservare le istruzioni indicate in questo manuale per garantire la sicurezza dell'utilizzatore;
- -La non osservanza delle indicazioni elencate in questo manuale comporterà l'annullamento della garanzia;
- -MECTRONICA S.r.l. non è responsabile per danni o lesioni causate dalla non osservanza delle informazioni elencate nel presente manuale.

## 2. SPECIFICHE

Il  GIRACASTAGNE  AUTOMATICO ROTOMARR entra in funzionamento grazie ad un alimentatore elettrico con le seguenti caratteristiche:

Voltaggio di alimentazione:

12 Volts DC

Amperag

# Image handling

In [3]:
import logging
import time
from pathlib import Path

from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

In [4]:
_log = logging.getLogger(__name__)

IMAGE_RESOLUTION_SCALE = 1.0

# Important: For operating with page images, we must keep them, otherwise the DocumentConverter
# will destroy them for cleaning up memory.
# This is done by setting PdfPipelineOptions.images_scale, which also defines the scale of images.
# scale=1 correspond of a standard 72 DPI image
# The PdfPipelineOptions.generate_* are the selectors for the document elements which will be enriched
# with the image field

In [5]:
def convert_page_into_image(conv_res, output_dir, doc_filename):
    """Converts each page of the document into an image and saves it."""
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")


In [6]:
def extract_images(conv_res, output_dir, doc_filename):
    """Extracts individual images (pictures) from the document and saves them."""
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

def extract_tables(conv_res, output_dir, doc_filename):
    """Extracts tables from the document and saves them as images."""
    table_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-table-{table_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

In [7]:
def process_pdf(file_path):
    """Main function to process the PDF document."""
    logging.basicConfig(level=logging.DEBUG)

    file_path = 'Brochure_PagoinRetePa.pdf'
    output_dir = Path("scratch")
    output_dir.mkdir(parents=True, exist_ok=True)

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()
    conv_res = doc_converter.convert(file_path)
    doc_filename = conv_res.input.file.stem

    #convert_page_into_image(conv_res, output_dir, doc_filename)
    extract_images(conv_res, output_dir, doc_filename)
    extract_tables(conv_res, output_dir, doc_filename)

    # Save markdown with embedded pictures
    md_filename = output_dir / f"{doc_filename}-with-images.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.EMBEDDED)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

    # Save HTML with externally referenced pictures
    html_filename = output_dir / f"{doc_filename}-with-image-refs.html"
    conv_res.document.save_as_html(html_filename, image_mode=ImageRefMode.REFERENCED)

    end_time = time.time() - start_time
    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")


In [10]:
def process_pdf(file_path):
    """Main function to process the PDF document."""
    logging.basicConfig(level=logging.DEBUG)

    file_path = 'Manuale ROTOMARR.pdf'
    output_dir = Path("scratch")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create a subfolder for this specific PDF file
    pdf_subfolder = output_dir / Path(file_path).stem
    pdf_subfolder.mkdir(parents=True, exist_ok=True)

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()
    conv_res = doc_converter.convert(file_path)
    doc_filename = conv_res.input.file.stem

    # Update the functions to save to the subfolder
    extract_images(conv_res, pdf_subfolder, doc_filename)
    extract_tables(conv_res, pdf_subfolder, doc_filename)

    # Save outputs to the subfolder as well
    md_filename = pdf_subfolder / f"{doc_filename}-with-images.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.EMBEDDED)

    md_filename = pdf_subfolder / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

    html_filename = pdf_subfolder / f"{doc_filename}-with-image-refs.html"
    conv_res.document.save_as_html(html_filename, image_mode=ImageRefMode.REFERENCED)

    end_time = time.time() - start_time
    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")

In [11]:
# Example usage:
process_pdf(file_path)

# Delete 'scratch' folder

In [1]:
import shutil
import os

def delete_folder(folder_path):
    """
    Deletes a folder and its contents, even if it's not empty.

    Args:
        folder_path (str): The path to the folder you want to delete.
    """
    try:
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting folder: {e}")

# Example usage:
folder_to_delete = "/content/scratch"  # Replace with the actual path
delete_folder(folder_to_delete)

Folder '/content/scratch' deleted successfully.


# [Continue] Text Processing, Chunking, Embedding, LLM

# Raw messy codes, dont look ;)

In [ ]:
from typing import List
from docling.document_converter import DocumentConverter  # For PDF content extraction
#from langchain_ollama.llms import OllamaLLM  # Local LLM integration
#from langchain.text_splitter import RecursiveCharacterTextSplitter  # For text chunking
#from langchain_huggingface import HuggingFaceEmbeddings  # For text embeddings
#from langchain_community.vectorstores import FAISS  # Vector database
#from langchain.chains import RetrievalQA  # For question-answering pipeline
#from langchain.prompts import PromptTemplate  # For customizing LLM prompts

In [ ]:
def extract_pdf_content(file_path) -> str:
    """
    Extract structured content from PDF using Docling library

    Args:
        file_path (str): Path to the PDF file

    Returns:
        str: Extracted content in markdown format
    """
    converter = DocumentConverter()
    result = converter.convert(file_path)
    return result.document.export_to_markdown()

In [ ]:
def create_vector_store(texts: List[str]) -> FAISS:
    """
    Create and initialize FAISS vector store using text embeddings

    Args:
        texts (List[str]): List of text chunks to be embedded

    Returns:
        FAISS: Initialized FAISS vector store containing embedded texts
    """
    # Initialize sentence transformer model for text embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L12-v2"
    )
    vector_store = FAISS.from_texts(texts, embeddings)
    return vector_store